In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Module 2. Векторная модель текста и классификация длинных текстов ###

**2.3.1**

In [ ]:
from sympy import diff, symbols, log, exp
x, y, w, b = symbols('x y w b')
y_hat = 1/(1+exp(-w*x-b))
eq = -y*log(y_hat)-(1-y)*log(1-y_hat)
print(diff(eq, w).simplify())

In [ ]:
import sympy.parsing.sympy_parser

sample_expr_str = 'x*(-y*exp(-b - w*x) - y + 1)/(exp(-b - w*x) + 1)'
sample_expr = sympy.parsing.sympy_parser.parse_expr(sample_expr_str)
sample_value = sample_expr.evalf(subs=dict(x=0.5, y=1, w=4, b=1))
print(sample_value)

**2.3.2**

In [ ]:
from sympy import diff, symbols, log, exp
x, y, w, b = symbols('x y w b')
y_hat = 1/(1+exp(-w*x-b))
eq = -y*log(y_hat)-(1-y)*log(1-y_hat)
print(diff(eq, b).simplify())

**2.3.3**

In [ ]:
from sympy import diff, symbols, log, exp
x, y, w, b, c = symbols('x y w b, c')
y_hat = 1/(1+exp(-w*x-b))
reg = c * (w**2 + b**2)
eq = -y*log(y_hat)-(1-y)*log(1-y_hat) + reg
print(diff(eq, w).simplify())

**2.3.6**

In [ ]:
import sys
import numpy as np

def parse_array(s):
    return np.array([int(s.strip()) for s in s.strip().split(' ')])

def read_array(inp):
    return parse_array(inp)

def calculate_pmi(a, b):
    #p(a,b) - вероятность того, что оба события совершились одновременно (и там и там единица).
    #p(a) , p(b) - вероятности реализации случайных событий.
    #pmi(a,b) = log(p(a,b)/(p(a)*p(b)))
    p_a_b = np.mean(a * b)
    p_a = np.mean(a)
    p_b = np.mean(b)
    return np.log(p_a_b / (p_a * p_b))  

a = read_array('1 0 0 1 1 0')
b = read_array('1 0 0 0 1 0')
pmi_value = calculate_pmi(a, b)

print('{:.6f}'.format(pmi_value))

**2.3.7**

In [ ]:
import numpy as np
s = 2
P = 10 / 10000 #заданная плотность распределения
Z = np.array([i**(-2) for i in range(1, 1001)]).sum()
rank = (1 / (Z * 10 / 10000))**(1/s)
num = int(1000 - rank) + 1
num

**2.4.3**

In [ ]:
import re
import sys

text = [
'Контактный телефон: 123123.',
'Что-нибудь надо придумать.',
'Значение числа Е=2.7182.',
'Демон123, как тебя зовут в реале?',
'-1-.15=-1.15',
'- 1 - .15 = -1.15',
'Какого ;%:?* тут происходит?'
]

# модифицируйте это регулярное выражение
parts = [
    r'[а-яё]+',
    r'-?\d*[.,]?\d+',
    r'\S'
]
#re.I -  re.IGNORECASE - Perform case-insensitive matching
#Флаг, избавляет нас от необходимости указывать заглавные буквы алфавита 
#при выделении слов и достаточно указать лишь диапазон строчных букв.
TOKENIZE_RE = re.compile(r'|'.join(parts), re.I) 


def tokenize(txt):
    return TOKENIZE_RE.findall(txt)


for line in text:
    print(' '.join(tokenize(line.strip().lower())))

**2.4.5**

In [ ]:
import re
import sys
import collections



def tokenize_text(text, lower=True):
    parts = [
        r'[\w\d]+'
    ]
    TOKENIZE_RE = re.compile(r'|'.join(parts), re.I)
    if lower:
        return TOKENIZE_RE.findall(text.lower())
    else:
        return TOKENIZE_RE.findall(text)

def tokenize_corpus(texts, lower=True):
    return [tokenize_text(text, lower=lower) for text in texts]

def build_vocabulary(tokenize_texts):
    word_counts = collections.defaultdict(int)
    
    for text in tokenize_texts:
        unique_tokens = set(text)
        for token in unique_tokens:
            word_counts[token] += 1
    doc_count = len(tokenize_texts)
    
    word_counts_sorted = sorted(word_counts.items(), reverse=False, key=lambda kv:(kv[1], kv[0]))
    
    word2id = {word: i for i, (word, _) in enumerate(word_counts_sorted)}
    word2freq = {word: word_count/doc_count for (word, word_count) in word_counts_sorted}
    
    return word2id, word2freq

texts = [
"Казнить нельзя, помиловать. Нельзя наказывать.",
"Казнить, нельзя помиловать. Нельзя освободить.",
"Нельзя не помиловать.",
"Обязательно освободить."
]

voc_word2id, voc_word2freq = build_vocabulary(tokenize_corpus(texts))

print(' '.join(voc_word2id.keys()))
print(' '.join(map(str, voc_word2freq.values())))